In [3]:
import numpy as np
import tensorflow as tf

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

2024-12-04 09:21:04.688251: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-04 09:21:04.706765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733304064.725048     225 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733304064.730459     225 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 09:21:04.750297: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
def create_model(input_size=200, learning_rate=0.002):
    inputs  = keras.Input(shape=(input_size, input_size, 3))
    conv    = keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    vectors = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv)
    flat    = keras.layers.Flatten()(vectors)
    inner   = keras.layers.Dense(64, activation='relu')(flat)
    output  = keras.layers.Dense(1, activation='sigmoid')(inner)

    model = keras.Model(inputs, output)

    optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.8)
    loss = keras.losses.BinaryCrossentropy()

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )

    return model

In [23]:
model = create_model()

In [8]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    './data/train',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

test_generator = datagen.flow_from_directory(
    './data/test',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [39]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'model_{epoch:02d}_{val_accuracy:.3f}.keras',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
)

In [40]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    callbacks=[checkpoint]
)


Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 15s 381ms/step - accuracy: 0.6923 - loss: 0.6121 - val_accuracy: 0.6567 - val_loss: 0.5976
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 15s 373ms/step - accuracy: 0.6882 - loss: 0.5922 - val_accuracy: 0.6716 - val_loss: 0.5899
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 360ms/step - accuracy: 0.6745 - loss: 0.6043 - val_accuracy: 0.6567 - val_loss: 0.6328
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 355ms/step - accuracy: 0.6992 - loss: 0.5961 - val_accuracy: 0.6368 - val_loss: 0.6301
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 15s 374ms/step - accuracy: 0.6766 - loss: 0.5806 - val_accuracy: 0.6965 - val_loss: 0.6132
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 353ms/step - accuracy: 0.7475 - loss: 0.5594 - val_accuracy: 0.6517 - val_loss: 0.6030
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 15s 363ms/step - accuracy: 0.6936 - loss: 0.5769 - val_accuracy: 0.6617 - val_loss: 0.5911
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 357ms/step - accuracy: 0.6572 - loss: 0.5989 - val_accu

In [41]:
history

In [42]:
history.history.items()

dict_items([('accuracy', [0.6862499713897705, 0.6700000166893005, 0.6812499761581421, 0.6600000262260437, 0.6800000071525574, 0.7074999809265137, 0.6850000023841858, 0.6725000143051147, 0.6899999976158142, 0.6625000238418579]), ('loss', [0.6052072644233704, 0.5982021689414978, 0.601817786693573, 0.6142834424972534, 0.5962433815002441, 0.5840916633605957, 0.578118085861206, 0.5926342606544495, 0.5926942229270935, 0.5917982459068298]), ('val_accuracy', [0.6567164063453674, 0.6716417670249939, 0.6567164063453674, 0.6368159055709839, 0.6965174078941345, 0.6517412662506104, 0.6616915464401245, 0.6019900441169739, 0.6965174078941345, 0.646766185760498]), ('val_loss', [0.597630500793457, 0.5899415016174316, 0.6328110694885254, 0.6300715804100037, 0.613237738609314, 0.603044867515564, 0.5910598039627075, 0.6358608603477478, 0.579459547996521, 0.6141744256019592])])

In [43]:
loss_list = history.history['val_loss']

In [44]:
np.mean(loss_list)

np.float64(0.6087291896343231)

In [22]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    './data/train',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

test_generator = datagen.flow_from_directory(
    './data/test',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [46]:
model = keras.models.load_model('model_05_0.697.keras')

In [47]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    callbacks=[checkpoint]
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 15s 367ms/step - accuracy: 0.6837 - loss: 0.5909 - val_accuracy: 0.6169 - val_loss: 0.6505
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 355ms/step - accuracy: 0.6828 - loss: 0.5962 - val_accuracy: 0.6468 - val_loss: 0.6106
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 15s 367ms/step - accuracy: 0.6541 - loss: 0.6172 - val_accuracy: 0.7463 - val_loss: 0.5758
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 354ms/step - accuracy: 0.6860 - loss: 0.5925 - val_accuracy: 0.6617 - val_loss: 0.6067
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 351ms/step - accuracy: 0.6842 - loss: 0.5856 - val_accuracy: 0.6866 - val_loss: 0.5887
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 358ms/step - accuracy: 0.6895 - loss: 0.5818 - val_accuracy: 0.7015 - val_loss: 0.5723
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 356ms/step - accuracy: 0.6666 - loss: 0.5807 - val_accuracy: 0.6517 - val_loss: 0.6187
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 356ms/step - accuracy: 0.7034 - loss: 0.5743 - val_accu

In [49]:
loss = history.history['val_loss']

In [50]:
np.mean(loss)

np.float64(0.5997341871261597)

In [52]:
acc = history.history['val_accuracy']

In [53]:
np.mean(acc)

np.float64(0.6711442768573761)